In [1]:
import requests
from bs4 import BeautifulSoup
import html5lib
#proxies = {'http': 'http://proxy.charite.de:8080',

#           'https': 'https://proxy.charite.de:8080'}
#print("Using HTTP proxy %s" % proxies['http'])

In [2]:
curURL = 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Spezial:MultiCategorySearch&limit=500&offset=0&wpInCategory1=Gl%C3%BCcksspielsucht'
s      = requests.session()
#r      = s.get(curURL,proxies=proxies)
r      = s.get(curURL)
soup   = BeautifulSoup(r.text,"html5lib")
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [3]:
print(soup.prettify()[0:100000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="de">
 <head>
  <meta charset="utf-8"/>
  <title>
   Hilfe finden! – Spielsucht-Hilfe-Berlin
  </title>
  <meta content="MediaWiki 1.25.2" name="generator"/>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <link href="/shb/opensearch_desc.php" rel="search" title="Spielsucht-Hilfe-Berlin (de)" type="application/opensearchdescription+xml"/>
  <link href="http://spielsucht-hilfe-berlin.de/shb/api.php?action=rsd" rel="EditURI" type="application/rsd+xml"/>
  <link href="/shb/index.php?title=Spezial:MultiCategorySearch" hreflang="x-default" rel="alternate"/>
  <link href="/shb/index.php?title=Spezial:Letzte_%C3%84nderungen&amp;feed=atom" rel="alternate" title="Atom-Feed für „Spielsucht-Hilfe-Berlin“" type="application/atom+xml"/>
  <link href="http://spielsucht-hilfe-berlin.de/shb/load.php?debug=false&amp;lang=de&amp;modules=mediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.sectionAnchor%7Cmediawiki.skinning.content.externallinks%7

In [7]:
from IPython.display import Image
url = 'http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png'
Image(url = url)

#from IPython.display import Image
#Image('http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png')

In [8]:
Image(url = 'http://www.cs.toronto.edu/~shiva/cscb07/img/dom/treeStructure.png')

In [4]:
links = soup.find_all("li")
print (type(links))
print(len(links))

<class 'bs4.element.ResultSet'>
142


In [5]:
print(links[0])

<li><a href="/shb/index.php?title=AG_Spielsucht" title="AG Spielsucht">AG Spielsucht</a></li>


In [6]:
contacts = {}
for element in links:
    contacts[element.a.get_text()] = {}
first_keys = list(contacts.keys())[0:9]
for key in first_keys:
    print(key)
    print(contacts[key])

AG Spielsucht
{}
AWO Potsdam
{}
Allgemeine Schuldnerberatung
{}
Anonyme Glücksspielsüchtige 55+ "Silvergambler
{}
Beratungsstelle für Suchtkranke in Treptow-Köpenick
{}
Café Beispiellos
{}
Creative Selbsthilfe Oase e.V. (CSO)
{}
Drogenhilfe Nord
{}
FAIRPLAY des KKH e.V.
{}


In [7]:
print(links[0].a["href"])
prefix = 'http://www.spielsucht-hilfe-berlin.de'

/shb/index.php?title=AG_Spielsucht


In [9]:
for element in links:
    contacts[element.a.get_text()]["link"] = prefix + element.a["href"]

In [10]:
for key in first_keys:
    print(key)
    print(contacts[key])

AG Spielsucht
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=AG_Spielsucht'}
AWO Potsdam
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=AWO_Potsdam'}
Allgemeine Schuldnerberatung
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Allgemeine_Schuldnerberatung'}
Anonyme Glücksspielsüchtige 55+ "Silvergambler
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Anonyme_Gl%C3%BCcksspiels%C3%BCchtige_55%2B_%22Silvergambler'}
Beratungsstelle für Suchtkranke in Treptow-Köpenick
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Beratungsstelle_f%C3%BCr_Suchtkranke_in_Treptow-K%C3%B6penick'}
Café Beispiellos
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Caf%C3%A9_Beispiellos'}
Creative Selbsthilfe Oase e.V. (CSO)
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Creative_Selbsthilfe_Oase_e.V._(CSO)'}
Drogenhilfe Nord
{'link': 'http://www.spielsucht-hilfe-berlin.de/

In [45]:
# now go the link and apply a function that reads out all the information
# do it with first one first
key = list(contacts.keys())[0:2][1]

#for key in first_keys:
print(key)
print(contacts[key])

# get the link
curLink = contacts[key]['link']
print(curLink)

s      = requests.session()
#r      = s.get(curLink,proxies=proxies)
r      = s.get(curLink)
ssoup  = BeautifulSoup(r.text,"html5lib")
print(type(ssoup))
print(ssoup.prettify()[0:100])

AWO Potsdam
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=AWO_Potsdam'}
http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=AWO_Potsdam
<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="de">
 <head>
  <meta charset="utf-8"/>
  <


In [66]:
# functions
def getLink(linkEntry):
    return(linkEntry['link'])

def getInfos(link):
    """Takes the link of a help institution and extracts
    name, address, contact"""
    s = requests.session()
    #r = s.get(curLink,proxies=proxies)
    r = s.get(link)
    ssoup = BeautifulSoup(r.text,"html5lib")
    infos = ssoup.find_all("td") # td means line of a table
    try:
        retInfos = [infos[0].get_text(),infos[3].get_text(),infos[5].get_text()]
    except:
        print('infos not in expected format')
        retInfos = ['NA','NA','NA']
    return(retInfos)

In [67]:
ID = []
name = []
address = []
contact = []

for counter, value in enumerate(contacts):
    print(counter)
    curLink = getLink(contacts[value])
    retInfos = getInfos(curLink)
    ID.append(value)
    name.append(retInfos[0])
    address.append(retInfos[1])
    contact.append(retInfos[2])ID

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
infos not in expected format
127
infos not in expected format
128
infos not in expected format
129
infos not in expected format
130
131
infos not in expected format
132
infos not in expected format
133
infos not in expected format
134
135
136
137
infos not in expected format


In [81]:
import pandas as pd
import openpyxl

#pd.DataFrame(ID)
#pd.DataFrame(name)

df = pd.DataFrame({'ID': ID, 'Name': name, 'Address': address,
             'Contact' : contact})

df.to_excel('Spielsucht_Hilfe.xlsx')

In [97]:
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# multi-line wrap
wb = load_workbook('Spielsucht_Hilfe.xlsx')
ws = wb.active # wb.active returns a Worksheet object

for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(wrapText=True)
    
    from openpyxl import Workbook

wb.save("Spielsucht_Hilfe_wrapped.xlsx")

#worksheet.cell('A1').style.alignment.wrap_text = True
#worksheet.cell('A1').value = "Line 1\nLine 2\nLine 3"

#workbook.save('wrap_text1.xlsx')
